In [1]:
import pandas as pd

загрузка датасета

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=19UAXN1hoh7xFcLVMVVk7gpSezH70jtJ7' -O database.db

--2024-04-14 20:25:13--  https://docs.google.com/uc?export=download&id=19UAXN1hoh7xFcLVMVVk7gpSezH70jtJ7
Resolving docs.google.com (docs.google.com)... 142.250.159.100, 142.250.159.101, 142.250.159.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.159.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=19UAXN1hoh7xFcLVMVVk7gpSezH70jtJ7&export=download [following]
--2024-04-14 20:25:13--  https://drive.usercontent.google.com/download?id=19UAXN1hoh7xFcLVMVVk7gpSezH70jtJ7&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.194.132, 2607:f8b0:4001:c10::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.194.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11632640 (11M) [application/octet-stream]
Saving to: ‘database.db’

database.db         100%[===================>]  11.09M  66.4MB/s  

In [3]:
import sqlite3

db_con = sqlite3.connect("./database.db")
cur = db_con.cursor()

In [4]:
cur.execute("""
    SELECT post_text
    FROM 'posts'
    ORDER BY post_id
""")

In [5]:
data = pd.DataFrame(cur.fetchall(), columns=['post_text'])

In [6]:
data

,post_text
0,None
1,🛢Нефть Вrent на лондонской бирже ICE впервые с...
2,❗️Омбудсмена Татьяну Москалькову госпитализиро...
3,🪶Путин подписал закон о внесудебной блокировке...
4,🤷🏻‍♂️После введения системы QR-кодов количеств...
...,...
15218,😦 В ЖК бизнес-класса «Крылья» обнаружили хосте...
15219,💳 Оформите карту UnionPay от Газпромбанка для ...
15220,😱 Три автомобиля попали в жёсткую аварию на 50...
15221,⚡️ Момент жуткой аварии попал на видео.


# remove emoji

In [7]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 2.7 MB/s eta 0:00:00


In [8]:
import emoji
def remove_emoji(text):
    return emoji.replace_emoji(text[0],'')

In [9]:
cdata = pd.DataFrame(data.apply(remove_emoji,axis=1))

In [10]:
cdata

,0
0,None
1,Нефть Вrent на лондонской бирже ICE впервые с ...
2,Омбудсмена Татьяну Москалькову госпитализирова...
3,Путин подписал закон о внесудебной блокировке ...
4,После введения системы QR-кодов количество при...
...,...
15218,В ЖК бизнес-класса «Крылья» обнаружили хостел...
15219,Оформите карту UnionPay от Газпромбанка для п...
15220,Три автомобиля попали в жёсткую аварию на 50-...
15221,Момент жуткой аварии попал на видео.


# lemmatization and stemming

In [11]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 894.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=23ebfeff35891dca7f44c8e32bef4954c50407fbcebc61aac949862ffef60976
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


часть скрипта для обработки текста

In [12]:
import pymorphy2
import re
morph = pymorphy2.MorphAnalyzer()

def remove_punctuation(word: str):
    return re.sub(r'[.,:;]', '', word)

def encode_num(word: str):
    if word.isnumeric():
        return "I" + str(len(word))
    elif word[1:].isnumeric():  # обработка чисел вида $*
        return word[0] + "I" + str(len(word) - 1)
    else: # обработка чисел вида *$
        return word[-1] + "I" + str(len(word) - 1)

def lemmatize(data):
    text = data[0]
    words = text.split() # разбиваем текст на слова
    res = []
    for word in words:
        if word.isnumeric() or word[1:].isnumeric() or word[:-1].isnumeric():
            word = encode_num(word)
            res.append(word)
        else:
            word = remove_punctuation(word)
            p = morph.parse(word)[0]
            res.append(p.normal_form)
    return res

In [13]:
tokenized_data = cdata.apply(lemmatize,axis=1)

In [14]:
tokenized_data = tokenized_data[1:]

In [15]:
tokenized_data[:10]

1        [нефть, вrent, на, лондонский, биржа, ice, впе...
2        [омбудсмен, татьяна, москальков, госпитализиро...
3        [путин, подписать, закон, о, внесудебный, блок...
4        [после, введение, система, qr-код, количество,...
5        [германия, с, I1, июль, ослабить, ковид-ограни...
                               ...                        
15218    [в, жк, бизнес-класс, «крылья», обнаружить, хо...
15219    [оформить, карта, unionpay, от, газпромбанк, д...
15220    [три, автомобиль, попасть, в, жёсткий, авария,...
15221         [момент, жуткий, авария, попасть, на, видео]
15222    [кто-то, в, это, год, себя, ооочень, хорошо, в...
Length: 15222, dtype: object

In [16]:
tokenized_list = tokenized_data.to_list()

#Обучение word2vec

In [17]:
from gensim.models.word2vec import Word2Vec
from gensim.utils import effective_n_jobs
model = Word2Vec(sentences = tokenized_list,
                 sg=1,
                 window=7,
                 vector_size = 10,
                 workers=effective_n_jobs(-1)-1,
                 epochs=10)

In [18]:
model.save("word2vec.model")

In [19]:
print(model.wv.most_similar(positive=['нефть'], topn=5))

[('фондовый', 0.9466264843940735), ('brent', 0.9419498443603516), ('$I2', 0.9372860789299011), ('рис', 0.9296262264251709), ('посещаемость', 0.9222639799118042)]


#Векторное представление всех текстов с помощью word2vec

In [ ]:
vectors_list = []
for i in range(len(tokenized_list)):
    vectors = [model.wv.get_vector(word) for word in tokenized_list if word in list(model.wv.index_to_key)]
    vectors_list.append(vectors)

In [ ]:
vectors_list

# Обучение Doc2Vec

In [21]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized_list)]
from gensim.utils import effective_n_jobs
doc2vec = Doc2Vec(documents=docs,vector_size =200,window = 7, epochs = 20, workers = effective_n_jobs(-1)-1)

In [22]:
doc2vec.save("doc2vec.model")

# векторное представление всех текстов с помощью doc2vec

In [23]:
vectors_doc = []
for i in range(len(tokenized_list)):
    if (i%1000==0):
        print(i)
    vectors_doc.append(doc2vec.infer_vector(tokenized_list[i]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


In [24]:
doc2vec.infer_vector(['москва'])

array([-0.00329588, -0.00594495, -0.0202322 ,  0.03938466,  0.01414455,
       -0.0280463 ,  0.0375769 ,  0.0519252 , -0.02335493,  0.02170836,
       -0.022143  ,  0.01350211, -0.00400629,  0.014706  , -0.00825438,
       -0.00376879, -0.04000959,  0.04260429,  0.01003891,  0.00244778,
        0.02889085, -0.02031985,  0.01798831,  0.01144196,  0.03433743,
        0.0020826 , -0.02456737, -0.00806956, -0.03568279, -0.00336068,
        0.01519743, -0.01023645, -0.00565378, -0.03448845,  0.01512667,
        0.06546372, -0.01959749,  0.02134753, -0.00067613, -0.03617102,
       -0.06305598, -0.02294918, -0.01249871,  0.00520147,  0.00779169,
       -0.01181386,  0.0098409 , -0.01561054,  0.01481789,  0.01776701,
        0.03874661, -0.0226563 ,  0.00209281, -0.02107118, -0.0090273 ,
        0.01899219,  0.0092001 , -0.04915913, -0.06019833, -0.00701201,
       -0.00767239,  0.01671219,  0.01148314,  0.02611559, -0.0153584 ,
       -0.0040076 ,  0.00272942,  0.06671619, -0.01078351,  0.06

In [25]:
tokenized_list

[['нефть',
  'вrent',
  'на',
  'лондонский',
  'биржа',
  'ice',
  'впервые',
  'с',
  'I2',
  'октябрь',
  'I4',
  'год',
  'подняться',
  'выше',
  '$I2',
  'за',
  'баррель'],
 ['омбудсмен', 'татьяна', 'москальков', 'госпитализировать', 'с', 'covid-19'],
 ['путин',
  'подписать',
  'закон',
  'о',
  'внесудебный',
  'блокировка',
  'ресурс',
  'содержать',
  '«порочить',
  'информацию»',
  'согласно',
  'текст',
  'документ',
  'пострадать',
  'должный',
  'написать',
  'заявление',
  'и',
  'подтвердить',
  'что',
  'информация',
  'опубликовать',
  'в',
  'интернет',
  '«порочить',
  'он',
  'честь',
  'и',
  'достоинство',
  'и',
  'связать',
  'с',
  'обвинение',
  'в',
  'совершение',
  'преступления»',
  'после',
  'десятидневный',
  'проверка',
  'прокуратура',
  'генпрокурор',
  'или',
  'он',
  'заместитель',
  'проверить',
  'обоснованность',
  'решение',
  'региональный',
  'прокуратура',
  'и',
  'передать',
  'дать',
  'в',
  'роскомнадзор',
  'с',
  'требование',
  'у

In [26]:
vectors_doc[:2]

[array([-2.55576465e-02, -5.96997188e-03, -1.32028013e-02,  7.69434273e-02,
         1.18200928e-02, -9.76150334e-02, -2.38338113e-03,  3.80759649e-02,
         1.22114713e-03,  1.51640810e-02,  1.27667282e-02, -2.20534373e-02,
         5.07084727e-02,  4.03473750e-02, -1.40452804e-02,  1.15314033e-03,
         5.97867817e-02, -3.83319482e-02,  1.04176616e-02, -1.75767094e-01,
         8.31402019e-02, -3.92128937e-02, -1.71530037e-03,  2.60291137e-02,
        -7.30743855e-02,  2.57788412e-02, -6.39552474e-02, -8.36327951e-03,
        -1.98730081e-02, -5.79739138e-02,  6.43351674e-02,  5.79797812e-02,
         2.27432661e-02,  3.69391181e-02,  2.41311789e-02, -1.47318663e-02,
        -1.49049545e-02, -3.42009217e-02, -5.61344884e-02, -1.22727826e-01,
        -2.28052363e-02, -8.95971619e-03, -7.86488503e-02,  2.00230312e-02,
         6.43393248e-02, -4.08643447e-02, -1.38972318e-02, -7.42703527e-02,
        -2.59455908e-02,  4.37300391e-02,  1.08174076e-02, -2.36465354e-02,
        -9.3

#Кластеризация HDBSCAN

In [27]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [28]:
import numpy as np

In [29]:
from sklearn.cluster import HDBSCAN

In [30]:
clustering_model = HDBSCAN(min_cluster_size = 5)

In [31]:
preds = clustering_model.fit_predict(vectors_doc)

In [32]:
pd.Series(preds).value_counts()

 0    15031
-1      186
 1        5
Name: count, dtype: int64

# Кластеризация DBSCAN

In [33]:
from sklearn.cluster import DBSCAN

In [34]:
dbscan_model = DBSCAN()

In [35]:
dbscan_preds = dbscan_model.fit_predict(vectors_doc)

In [36]:
pd.Series(dbscan_preds).value_counts()

-1    12731
 0     2485
 1        6
Name: count, dtype: int64

# Кластеризация K-Means + silhouette

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
kmeans_preds_list = []
for clusters in range(2,20):
    kmeans_model = KMeans(n_clusters=clusters)
    kmeans_preds = kmeans_model.fit_predict(vectors_doc)
    kmeans_preds_list.append(kmeans_preds)
    print("try following number of clusters: ",clusters,'silhouette score = ', silhouette_score(X = vectors_doc,labels = kmeans_preds))

# try PCA or T-SNE to visualize vectors

https://github.com/aniketbote/Document-Clustering-Doc2vec/blob/master/Clustering/Clustering_code_Doc2Vec/clustering_Documents_Doc2Vec.py

In [ ]:
!pip install matplotlib scikit-learn numpy

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
flattened_vectors = [vec for sublist in vectors_list for vec in sublist]
flattened_vectors = np.array(flattened_vectors)

In [ ]:
flattened_vectors = np.unique(flattened_vectors, axis=0)

In [ ]:
tsne = TSNE(n_components=2, random_state=0)

In [ ]:
vectors_2d = tsne.fit_transform(flattened_vectors)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1])
plt.show()

In [ ]:
doc2vec_vectors_np = np.array(vectors_doc)

tsne_doc2vec = TSNE(n_components=2, random_state=0)
reduced_doc2vec_vectors = tsne_doc2vec.fit_transform(doc2vec_vectors_np)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(reduced_doc2vec_vectors[:, 0], reduced_doc2vec_vectors[:, 1])
plt.title('t-SNE Visualization of Doc2Vec Vectors')
plt.show()